In [40]:
import webvtt
import jiwer

First, we'll use the Python webvtt library (https://webvtt-py.readthedocs.io/en/latest/usage.html) to read in a sample .vtt file. With each caption line, it parses out the start time, end time, and caption text. But here, we just want the caption text.

In [41]:
original_caption_file = 'original_vtt_files/CFCH-SFF_2014_0629_China_Teahouse_Commons_0004-000001.vtt'
caption_list = [caption.text for caption in webvtt.read(original_caption_file)]
caption_list[:5]

["Speaker 1: Hello, hello everyone we're getting set up for our next session in Teahouse Commons.",
 'Speaker 1: You are in the China Program of the 2014 Smithsonian Folklife Festival.',
 'Speaker 1: This program is presented through a collaboration by the Center for Folklife and Cultural Heritage at the Smithsonian with the China International Cultural Association, working with the China Arts and Entertainment Group in Beijing.',
 "Speaker 1: Thanks a lot for being here. Today I don't know if you've noticed but we're doing programs slightly different today.",
 'Speaker 1: The China Festival program actually brought over 120 artists and presenters from China to participate in this program.']

For the purposes of our error rate calculator, we join all of the individual lines of text together with a space to make 1 really long string.

In [42]:
transcription_full_text = ' '.join(caption_list)
transcription_full_text[:250]

"Speaker 1: Hello, hello everyone we're getting set up for our next session in Teahouse Commons. Speaker 1: You are in the China Program of the 2014 Smithsonian Folklife Festival. Speaker 1: This program is presented through a collaboration by the Cen"

Then we do the same thing with the .vtt file generated by Whisper...

In [43]:
whisper_caption_file = 'whisper_small_vtt/CFCH-SFF_2014_0629_China_Teahouse_Commons_0004-000001.vtt'
whisper_caption_list = [caption.text for caption in webvtt.read(whisper_caption_file)]
whisper_caption_list[:5]

["Hello, hello everyone. We're getting set up for our next session in tea house comments. You are in the",
 'China program of the 2014 Smithsonian Folklife Festival',
 'This program is presented through a collaboration by the Center for Folklife and Cultural Heritage at the Smithsonian with the China',
 'International Culture Association',
 'Working with the China Arts and Entertainment Group in Beijing. Thanks a lot for being here today']

In [44]:
whisper_full_text = ' '.join(whisper_caption_list)
whisper_full_text[:250]

"Hello, hello everyone. We're getting set up for our next session in tea house comments. You are in the China program of the 2014 Smithsonian Folklife Festival This program is presented through a collaboration by the Center for Folklife and Cultural H"

Now we use the jiwer Python library (https://github.com/jitsi/jiwer) to align the 2 full transcriptions. For the initial assessment, we see a 36.87% WER (or word error rate).

In [45]:
def limit_output_width(input_string):
    WIDTH = 125
    lines = input_string.splitlines()
    new_lines = [line[:WIDTH] for line in lines]
    return '\n'.join(new_lines)

In [46]:
out = jiwer.process_words(transcription_full_text, whisper_full_text)
print(limit_output_width(jiwer.visualize_alignment(out)))

sentence 1
REF: Speaker 1: Hello, hello  everyone we're getting set up for our next session in Teahouse Commons.   Speaker 1: You are in
HYP: ******* ** Hello, hello everyone. We're getting set up for our next session in      tea    house comments. ** You are in
           D  D                      S     S                                               S        S         S  D           

number of sentences: 1
substitutions=1665 deletions=584 insertions=355 hits=4813

mer=35.11%
wil=51.99%
wip=48.01%
wer=36.87%


It looks like a lot of those errors come from mismatches in upper/lower case, and punctuations. Let's try converting both to lower case first. It looks like that drops the WER to 31.7%.

In [47]:
out = jiwer.process_words(transcription_full_text.lower(), whisper_full_text.lower())
print(limit_output_width(jiwer.visualize_alignment(out)))

sentence 1
REF: speaker 1: hello, hello  everyone we're getting set up for our next session in teahouse commons.   speaker 1: you are in
HYP: ******* ** hello, hello everyone. we're getting set up for our next session in      tea    house comments. ** you are in
           D  D                      S                                                     S        S         S  D           

number of sentences: 1
substitutions=1292 deletions=588 insertions=359 hits=5182

mer=30.17%
wil=44.35%
wip=55.65%
wer=31.70%


And now with some regular expression help (that I found from Stack Overflow), we can remove the punctuations from both strings. Now we're down to 20.35% WER. Not too bad. 

Especially since it looks like most of the remaining errors come from Speaker Tags that are left in from the original transcription. If we can remove those, I'm sure we would get a much better rate.

In [48]:
import re, string

In [49]:
regex = re.compile('[%s]' % re.escape(string.punctuation))

In [50]:
out = jiwer.process_words(regex.sub('', transcription_full_text.lower()), 
                          regex.sub('', whisper_full_text.lower()))
print(limit_output_width(jiwer.visualize_alignment(out)))

sentence 1
REF: speaker 1 hello hello everyone were getting set up for our next session in teahouse commons  speaker 1 you are in the ch
HYP: ******* * hello hello everyone were getting set up for our next session in      tea   house comments * you are in the ch
           D D                                                                         S       S        S D                  

number of sentences: 1
substitutions=473 deletions=582 insertions=377 hits=5983

mer=19.31%
wil=25.56%
wip=74.44%
wer=20.35%
